In [1]:
# 3種模型 vgg16 , resnet50, densenet121
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input as preprocess_input_densenet
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input as preprocess_input_inceptionresnetv2

from keras.preprocessing import image
import numpy as np
from numpy import linalg as LA
import os
import h5py
# model = VGG16(weights='imagenet', include_top=False)
# img_path = './images2/089037468.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
# features = model.predict(x)

In [2]:
class VGGNet:
    def __init__ (self):
        self.input_shape = (224,224,3)
        self.weight = 'imagenet'
        self.pooling = 'max'
        # VGG16
        self.model_vgg = VGG16(weights=self.weight,
                              input_shape = (self.input_shape[0],self.input_shape[1], self.input_shape[2]),
                              pooling = self.pooling, include_top = False)
        self.model_vgg.predict(np.zeros((1,224,224,3)))
        
        # ResNet50
        self.model_resnet = ResNet50(weights = self.weight, 
                                     input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]),
                                     pooling = self.pooling, include_top = False)
        self.model_resnet.predict(np.zeros((1, 224, 224, 3)))
        
        # DenseNet121
        self.model_densenet = DenseNet121(weights = self.weight, 
                                          input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]), 
                                          pooling = self.pooling, include_top = False)
        self.model_densenet.predict(np.zeros((1, 224, 224, 3)))
        
#         keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights='imagenet',
#                                                                  input_tensor=None, input_shape=None, 
#                                                                  pooling=None, classes=1000)
        # DenseNet121
        self.model_inceptionresnetv2 = InceptionResNetV2(weights = self.weight, 
                                          input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]), 
                                          pooling = self.pooling, include_top = False)
        self.model_inceptionresnetv2.predict(np.zeros((1, 224, 224, 3)))
        
    # 提取VGG16最後一層conz特徵
    def vgg_extract_feat(self, img_path):
        img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img = image.img_to_array(img)
        img = np.expand_dims(img,axis=0)
        img = preprocess_input_vgg(img)
        feat = self.model_vgg.predict(img)
        norm_feat = feat[0] / LA.norm(feat[0])
        return norm_feat
    
     #提取resnet50最後一層卷積特徵
    def resnet_extract_feat(self, img_path):
        img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input_resnet(img)
        feat = self.model_resnet.predict(img)
        # print(feat.shape)
        norm_feat = feat[0]/LA.norm(feat[0])
        return norm_feat
    
    #提取densenet121最後一層卷積特徵
    def densenet_extract_feat(self, img_path):
        img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input_densenet(img)
        feat = self.model_densenet.predict(img)
        # print(feat.shape)
        norm_feat = feat[0]/LA.norm(feat[0])
        return norm_feat
        #提取densenet121最後一層卷積特徵
    def inceptionresnetv2_extract_feat(self, img_path):
        img = image.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input_densenet(img)
        feat = self.model_densenet.predict(img)
        # print(feat.shape)
        norm_feat = feat[0]/LA.norm(feat[0])
        return norm_feat

In [3]:
def get_imlist(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

In [4]:
if __name__=="__main__":
    database = 'images2'
    index = 'models/vgg_featureCNN.h5'
    img_list = get_imlist(database)
    

    print("--------------------------------------------------")
    print("               feature extraction starts ")
    print("--------------------------------------------------")
    
    
    feats = []
    names = []
    
    model = VGGNet()
    for i, img_path in enumerate(img_list): #修改此處改變提取特徵的網路
        
        # VGG16
#         norm_feat = model.vgg_extract_feat(img_path)
        
        # ResNet50
#         norm_feat = model.resnet_extract_feat(img_path)
        
        # DenseNet121
        norm_feat = model.densenet_extract_feat(img_path)
        # inceptionresnetv
        norm_feat = model.inceptionresnetv2_extract_feat(img_path)
        
        
        img_name = os.path.split(img_path)[1]
        feats.append(norm_feat)
        names.append(img_name)
        print('extracting featurn from image NO. %d , %d images in total' %((i + 1),len(img_list)))
        
    feats = np.array(feats)
    output = index
    
    print("--------------------------------------------------")
    print("               writing feature extraction results")
    print("--------------------------------------------------")
    
    h5f = h5py.File(output, 'w')
    h5f.create_dataset('dataset_1',data = feats)
    h5f.create_dataset('dataset_2',data = np.string_(names))
    
    h5f.close()
    model.save("post_h5f")

--------------------------------------------------
               feature extraction starts 
--------------------------------------------------
extracting featurn from image NO. 1 , 106 images in total
extracting featurn from image NO. 2 , 106 images in total
extracting featurn from image NO. 3 , 106 images in total
extracting featurn from image NO. 4 , 106 images in total
extracting featurn from image NO. 5 , 106 images in total
extracting featurn from image NO. 6 , 106 images in total
extracting featurn from image NO. 7 , 106 images in total
extracting featurn from image NO. 8 , 106 images in total
extracting featurn from image NO. 9 , 106 images in total
extracting featurn from image NO. 10 , 106 images in total
extracting featurn from image NO. 11 , 106 images in total
extracting featurn from image NO. 12 , 106 images in total
extracting featurn from image NO. 13 , 106 images in total
extracting featurn from image NO. 14 , 106 images in total
extracting featurn from image NO. 15 , 

AttributeError: 'File' object has no attribute 'save'

In [ ]:
#  test.py
# -*- coding: utf-8 -*-
# from extract_cnn_vgg16_keras import VGGNet
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# import argparse

query = './database/089037524.jpg'
index = 'models/vgg_featureCNN.h5'
result = 'images2'
# read in indexed images' feature vectors and corresponding image names
h5f = h5py.File(index,'r')
# feats = h5f['dataset_1'][:]
feats = h5f['dataset_1'][:]
print(feats)
imgNames = h5f['dataset_2'][:]
print(imgNames)
h5f.close()
        
print("--------------------------------------------------")
print("               searching starts")
print("--------------------------------------------------")
    
# read and show query image
# queryDir = args["query"]
queryImg = mpimg.imread(query)
plt.title("Query Image")
plt.imshow(queryImg)
plt.show()

# init VGGNet16 model
model = VGGNet()

# extract query image's feature, compute simlarity score and sort #修改此處改變提取特徵的網路

# VGG16
# queryVec = model.vgg_extract_feat(query)

# ResNet50
# queryVec = model.resnet_extract_feat(query)

# DenseNet121
# queryVec = model.densenet_extract_feat(query)

# Inceptionresnetv2
queryVec = model.inceptionresnetv2_extract_feat(query)

# print(queryVec.shape)
# print(feats.shape)
scores = np.dot(queryVec, feats.T)
rank_ID = np.argsort(scores)[::-1]
rank_score = scores[rank_ID]
# print (rank_ID)
print (rank_score)


# number of top retrieved images to show
maxres = 3          #檢索出三張相似度最高的圖片
imlist = []
for i,index in enumerate(rank_ID[0:maxres]):
    imlist.append(imgNames[index])
    # print(type(imgNames[index]))
    print("image names: "+str(imgNames[index]) + " scores: %f"%rank_score[i])
print("top %d images in order are: " %maxres, imlist)
# show top #maxres retrieved result one by one
for i,im in enumerate(imlist):
    image = mpimg.imread(result+"/"+str(im, 'utf-8'))
    plt.title("search output %d" %(i+1))
    plt.imshow(image)
    plt.show()